## Generate Random pairs of choices of course by students to be taken in Slot A and SLot B
- Courses that are in same slots follow same schedule 
- A student cannot take more than one course from the same slot.
- A student in satisfied if both of his selected courses are in differnt slots

In [2]:
import random

def generate_pairs(n):
    values = [0, 1, 2, 3, 5,6,7]
    pairs = []
    
    for _ in range(n):
        a, b = random.sample(values, 2)  # Ensures a != b
        pairs.append((a, b))
    
    return pairs

# Example usage
n = 20
print(generate_pairs(n))


[(2, 3), (1, 0), (7, 5), (6, 5), (0, 7), (2, 6), (3, 2), (7, 1), (1, 3), (1, 6), (7, 2), (1, 5), (1, 6), (7, 6), (7, 5), (5, 0), (2, 6), (0, 3), (1, 5), (7, 3)]


- Non-Flexible means : Number of electives per slot = Number of electives/2
- Flexible means Number of electives per slot need not be same for both

## PULP LIBRARY (BRANCH AND BOUND METHOD) (NOT FLEXIBLE)

In [5]:
import pulp

# Function to solve the elective assignment problem
def assign_electives(students, electives, k):
    # Number of students and electives
    N = len(students)
    M = len(electives)

    # Create a linear programming problem
    problem = pulp.LpProblem("Elective_Assignment_Problem", pulp.LpMaximize)

    # Decision variables
    a = pulp.LpVariable.dicts("SlotA", range(M), cat='Binary')  # Slot A assignments
    b = pulp.LpVariable.dicts("SlotB", range(M), cat='Binary')  # Slot B assignments
    S = pulp.LpVariable.dicts("Satisfaction", range(N), cat='Binary')  # Student satisfaction

    # Objective function: Maximize the number of satisfied students
    problem += pulp.lpSum(S[i] for i in range(N)), "Maximize_Satisfaction"

    # Constraint: Equal number of electives in Slot A and Slot B
    problem += pulp.lpSum(a[j] for j in range(M)) == k, "Equal_SlotA"
    problem += pulp.lpSum(b[j] for j in range(M)) == k, "Equal_SlotB"

    # Constraint: An elective can only be assigned to one of the two slots
    for j in range(M):
        problem += a[j] + b[j] <= 1, f"One_Slot_{j}"

    # Satisfaction constraints for students
    for i in range(N):
        elective1, elective2 = students[i]
        problem += S[i] <= a[elective1] + (1 - b[elective2]), f"Satisfaction_Condition_1_{i}"
        problem += S[i] <= (1 - a[elective1]) + b[elective2], f"Satisfaction_Condition_2_{i}"

    # Solve the problem
    problem.solve()

    # Print results
    print(f"Status: {pulp.LpStatus[problem.status]}")
    satisfied_students = [i for i in range(N) if S[i].varValue == 1]
    print(f"Number of satisfied students: {len(satisfied_students)}")
    print(f"Total students = {N}")
    print("Satisfied students indices:", satisfied_students)
    
    slot_a_electives = [electives[j] for j in range(M) if a[j].varValue == 1]
    slot_b_electives = [electives[j] for j in range(M) if b[j].varValue == 1]
    
    print("Electives in Slot A:", slot_a_electives)
    print("Electives in Slot B:", slot_b_electives)

# Example data
students = generate_pairs(100)
electives = [0, 1, 2, 3, 4, 5,6,7]  # List of electives
k = 4  # Number of electives per slot

# Run the function
assign_electives(students, electives, k)


Status: Optimal
Number of satisfied students: 66
Total students = 100
Satisfied students indices: [0, 1, 2, 4, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 20, 22, 27, 28, 29, 31, 32, 34, 37, 38, 39, 41, 42, 44, 46, 48, 49, 50, 53, 55, 56, 57, 59, 60, 63, 64, 65, 66, 67, 68, 69, 70, 72, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 86, 88, 90, 91, 92, 93, 94, 97, 98]
Electives in Slot A: [2, 4, 5, 7]
Electives in Slot B: [0, 1, 3, 6]


## Using Barrier penalty and hit and trial (FLEXIBLE)

In [6]:
import math
from itertools import combinations

# Function to calculate satisfaction and barrier penalty with the new constraint
def calculate_satisfaction_and_barrier(students, electives, slot_a, slot_b, t):
    total_barrier = 0
    satisfied_students = 0

    # Constraint: We want the number of subjects in Slot A and Slot B to be as close as possible
    # The penalty increases with the absolute difference between the sizes of Slot A and Slot B
    total_barrier += -t * math.log(abs(len(slot_a) - len(slot_b)) + 1)

    # Penalty for constraint: No elective can be in both Slot A and Slot B
    overlap = set(slot_a) & set(slot_b)
    if len(overlap) > 0:
        total_barrier += -t * math.log(len(overlap) + 1)

    # Calculate student satisfaction
    for student in students:
        elective1, elective2 = student
        if (elective1 in slot_a and elective2 in slot_b) or (elective1 in slot_b and elective2 in slot_a):
            satisfied_students += 1

    return satisfied_students, total_barrier

# Barrier method optimization for elective assignment with flexible slot sizes
def assign_electives_barrier_no_library(students, electives, k, initial_t=1000, mu=0.5, epsilon=1e-4):
    N = len(electives)  # Total number of electives

    # Possible ranges for the number of electives in Slot A and Slot B
    min_slot_size = N // 2  # Minimum number of electives in any slot
    max_slot_size = N - min_slot_size  # Maximum number of electives in any slot

    best_satisfaction = -float('inf')  # Track the best satisfaction score
    best_assignment = None  # Track the best assignment (Slot A, Slot B)
    t = initial_t  # Initialize barrier parameter

    while t > epsilon:

        # Try different slot sizes for Slot A (and Slot B will automatically be N - Slot A size)
        for slot_size_a in range(min_slot_size, max_slot_size + 1):
            # All possible combinations of `slot_size_a` electives for Slot A
            slot_a_combinations = list(combinations(electives, slot_size_a))

            # Try every possible combination for Slot A of size `slot_size_a`
            for slot_a in slot_a_combinations:
                slot_a_set = set(slot_a)
                # Slot B should contain all remaining electives
                slot_b = [e for e in electives if e not in slot_a_set]

                # Calculate satisfaction and barrier penalty for this assignment
                satisfaction, barrier = calculate_satisfaction_and_barrier(students, electives, slot_a, slot_b, t)
                total_score = satisfaction + barrier

                # Update best score and assignment if we found a better one
                if total_score > best_satisfaction:
                    best_satisfaction = total_score
                    best_assignment = (slot_a, slot_b)

        # Reduce the barrier parameter t
        t *= mu
        if(t<epsilon):
            print(f"Cutoff at {t}")

    # Print the results
    print(f"Best Satisfaction: {best_satisfaction}")
    print(f"Slot A Electives: {best_assignment[0]} (Total: {len(best_assignment[0])})")
    print(f"Slot B Electives: {best_assignment[1]} (Total: {len(best_assignment[1])})")

    # Calculate the number of satisfied students
    satisfied_students = 0
    for student in students:
        elective1, elective2 = student
        if (elective1 in best_assignment[0] and elective2 in best_assignment[1]) or (elective1 in best_assignment[1] and elective2 in best_assignment[0]):
            satisfied_students += 1

    print(f"Number of satisfied students: {satisfied_students} out of {len(students)}")

# Example data
students = generate_pairs(500)  # Each tuple represents (elective1, elective2)
electives = [0,1,2,3,5,6,7]  # List of electives
k = 2  # Number of electives per slot (flexible)

# Run the function
assign_electives_barrier_no_library(students, electives, k)


Cutoff at 5.9604644775390625e-05
Best Satisfaction: 298.99991737041705
Slot A Electives: (0, 2, 7) (Total: 3)
Slot B Electives: [1, 3, 5, 6] (Total: 4)
Number of satisfied students: 299 out of 500
